In [1]:
import requests
import re
import os

download = False
filename = 'slides-pronperovskites.html'
max_url_length = 512 #chars
url_string = '\"http'
cdn_string = 'amazonaws'

out_filename = "index.html"

src_foldername = 'src'


In [2]:
with open(filename,'r') as f:
    s = f.read()
    url_pos_list = []
    for m in re.finditer('\"http',s):
        url_pos = m.start()
        start_of_url_pos = url_pos+1
        end_of_url_pos = s[start_of_url_pos:start_of_url_pos+max_url_length].find('\"')+start_of_url_pos
        url_pos_list.append((start_of_url_pos,end_of_url_pos))

In [3]:
urls = [s[url_pos[0]:url_pos[1]] for url_pos in url_pos_list]
content_urls = [(url,url_pos_tup) for url, url_pos_tup in zip(urls,url_pos_list) if cdn_string in url]

In [4]:
try:
    os.mkdir(src_foldername)
except FileExistsError:
    pass

In [5]:
s_prev = 0
new_file =""
for url, pos in content_urls:

    local_path_file = os.path.join(src_foldername,*url.split('/')[3:])
    local_path = os.path.join(src_foldername,*url.split('/')[3:-1])
    try:
        os.makedirs(local_path)
    except FileExistsError:
        pass
    if download:
        r = requests.get(url, allow_redirects=True)
        with open(local_path_file,'wb') as f:
            f.write(r.content)
    new_file += s[s_prev:pos[0]]
    new_file += local_path_file
    s_prev = pos[1]
new_file += s[s_prev:]

In [6]:
with open(out_filename, 'w') as f:
    f.write(new_file)